まずは、データセットを読み込む。

In [1]:
def load_data_and_labels(filename):
    sents, labels = [], []
    with open(filename) as f:
        words, tags = [], []
        for line in f:
            line = line.rstrip()
            if len(line) == 0 or line.startswith('-DOCSTART-'):
                if len(words) != 0:
                    sents.append(words)
                    labels.append(tags)
                    words, tags = [], []
            else:
                word, tag = line.split('\t')
                words.append(word)
                tags.append(tag)
    return sents, labels

sents, labels = load_data_and_labels('data/dataset.tsv')

In [2]:
print(sents[1])
print(labels[1])

['こ', 'の', '記', '事', 'で', 'は', '言', '語', '（', 'げ', 'ん', 'ご', '）', '、', '特', 'に', '自', '然', '言', '語', 'に', 'つ', 'い', 'て', '述', 'べ', 'る', '。']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-product/language/language_other', 'I-product/language/language_other', 'I-product/language/language_other', 'I-product/language/language_other', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


上記に示したように読み込んだデータセットでは文字単位でラベルがついている。今回作るベースラインでは単語単位で認識したいので、単語レベルにラベルを付け直すことにする。

タスクとしては以下の通り。
* 文字のリストを結合して文字列にする
* 文字列を形態素解析器で解析し、分かち書きする
* 分かち書きした単語のリストに対してラベルを付け直す。

In [3]:
docs = [''.join(sent) for sent in sents]
docs[1]

'この記事では言語（げんご）、特に自然言語について述べる。'

In [7]:
import MeCab
t = MeCab.Tagger()


def tokenize(sent):
    tokens = []
    t.parse('')  # for UnicodeDecodeError
    node = t.parseToNode(sent)

    while node:
        feature = node.feature.split(',')
        surface = node.surface    # 表層形
        pos = feature[0]          # 品詞
        tokens.append((surface, pos))
        node = node.next

    return tokens[1:-1]

tokenized_docs = [[d[0] for d in tokenize(doc)] for doc in docs]
poses = [[d[1] for d in tokenize(doc)] for doc in docs]
# tokenized_docs = [tokenize(doc) for doc in docs]
# tokenized_docs = [[d[0] for d in doc] for doc in tokenized_docs]
# poses = [[d[1] for d in doc] for doc in tokenized_docs]
# tokenized_docs[1]
# poses[1]

分かち書きまではできた。その後が面倒くさい。どうやってラベルを単語単位で付け直すか？文字と単語の対応でも取るか。
1. 形態素を1つ取り出す
2. 形態素を構成するラベルを文字列マッチングによって取り出す
3. ラベルを修正する

In [10]:
doc = docs[1]
i = 0
for word in tokenized_docs[1]:
    j = len(word)
    print('{}\t{}'.format(word, labels[1][i: i+j][0]))
    i += j

この	O
記事	O
で	O
は	O
言語	O
（	O
げん	O
ご	O
）	O
、	O
特に	O
自然	B-product/language/language_other
言語	I-product/language/language_other
について	O
述べる	O
。	O


ラベルの修正でかなりいい加減な処理をしているが、ひとまずこれで良しとしよう。ではこれをすべての文書に対して行ってやる。

In [11]:
for t_doc, label in zip(tokenized_docs, labels):
    i = 0
    for word in t_doc:
        j = len(word)
        tag = label[i: i+j][0]
        print('{}\t{}'.format(word, tag))
        i += j
    break

アンパサンド	O
(,	O
&)	O
と	O
は	O
「	O
…	O
と	O
…	O
」	O
を	O
意味	O
する	O
記号	O
で	O
ある	B-product/title/position_vocation
。	O
ラテン語	O
の	I-product/language/language_other
の	I-product/language/language_other
合	I-product/language/language_other
字	O
で	O
、	O
Trebuchet	B-location/gpe/city
MS	I-product/weapon
フォント	I-product/weapon
で	I-product/weapon
は	O
、	O
と	O
表示	O
さ	O
れ	O
"	O
et	O
"	O
の	O
合	O
字	O
で	O
ある	O
こと	O
が	O
容易	O
に	O
わかる	O
。	O
ampersa	O
、	O
すなわち	O
"	O
and	O
per	O
se	O
and	O
"、	O
その	O
意味	O
は	O
"	O
and	O
[	O
the	O
symbol	O
which	O
]	O
by	O
itself	O
[	O
is	O
]	O
and	O
"	O
で	O
ある	O
。	O


むむっ、誤った対応付を行っている。原因を分析するために元の対応付を見てみる。

In [12]:
for ch, label in zip(sents[0], labels[0]):
    print('{}\t{}'.format(ch, label))

ア	O
ン	O
パ	O
サ	O
ン	O
ド	O
 	O
(	O
,	O
 	O
&	O
)	O
 	O
と	O
は	O
「	O
…	O
と	O
…	O
」	O
を	O
意	O
味	O
す	O
る	O
記	B-product/title/position_vocation
号	I-product/title/position_vocation
で	O
あ	O
る	O
。	O
ラ	B-product/language/language_other
テ	I-product/language/language_other
ン	I-product/language/language_other
語	I-product/language/language_other
の	O
 	O
の	O
合	B-location/gpe/city
字	I-location/gpe/city
で	O
、	O
T	B-product/weapon
r	I-product/weapon
e	I-product/weapon
b	I-product/weapon
u	I-product/weapon
c	I-product/weapon
h	I-product/weapon
e	I-product/weapon
t	I-product/weapon
 	I-product/weapon
M	I-product/weapon
S	I-product/weapon
フ	O
ォ	O
ン	O
ト	O
で	O
は	O
、	O
と	O
表	O
示	O
さ	O
れ	O
 	O
"	O
e	O
t	O
"	O
 	O
の	O
合	O
字	O
で	O
あ	O
る	O
こ	O
と	O
が	O
容	O
易	O
に	O
わ	O
か	O
る	O
。	O
a	O
m	O
p	O
e	O
r	O
s	O
a	O
、	O
す	O
な	O
わ	O
ち	O
 	O
"	O
a	O
n	O
d	O
 	O
p	O
e	O
r	O
 	O
s	O
e	O
 	O
a	O
n	O
d	O
"	O
、	O
そ	O
の	O
意	O
味	O
は	O
"	O
a	O
n	O
d	O
 	O
[	O
t	O
h	O
e	O
 	O
s	O
y	O
m	O
b	O
o	O
l	O
 	O
w	O
h	O
i	O
c	O
h	O
]	O
 	O
b	O


どうやら元の文字列にはあった空白が形態素解析で消えたことで、対応付にずれが生じたのが原因のようだ。単純に対応付けるのではなく、単語が始まるかどうかで補正をする。

In [14]:
tags = []
for t_doc, doc, label in zip(tokenized_docs, docs, labels):
    i = 0
    doc_tags = []
    for word in t_doc:
        j = len(word)
        while not doc[i:].startswith(word):  # correct
            i += 1
        tag = label[i: i+j][0]
        # print('{}\t{}'.format(word, tag))
        doc_tags.append(tag)
        i += j
    tags.append(doc_tags)
    # break

In [17]:
sents, labels = [], []

In [33]:
data = []
i = 0
for doc, poss, labels in zip(tokenized_docs, poses, tags):
    if i == 10000:
        break
    data.append([(w, p, l) for w, p, l in zip(doc, poss, labels)])
    i += 1

In [34]:
data[0]

[('アンパサンド', '名詞', 'O'),
 ('(,', '名詞', 'O'),
 ('&)', '名詞', 'O'),
 ('と', '助詞', 'O'),
 ('は', '助詞', 'O'),
 ('「', '記号', 'O'),
 ('…', '記号', 'O'),
 ('と', '助詞', 'O'),
 ('…', '記号', 'O'),
 ('」', '記号', 'O'),
 ('を', '助詞', 'O'),
 ('意味', '名詞', 'O'),
 ('する', '動詞', 'O'),
 ('記号', '名詞', 'B-product/title/position_vocation'),
 ('で', '助動詞', 'O'),
 ('ある', '助動詞', 'O'),
 ('。', '記号', 'O'),
 ('ラテン語', '名詞', 'B-product/language/language_other'),
 ('の', '助詞', 'O'),
 ('の', '助詞', 'O'),
 ('合', '名詞', 'B-location/gpe/city'),
 ('字', '名詞', 'I-location/gpe/city'),
 ('で', '助詞', 'O'),
 ('、', '記号', 'O'),
 ('Trebuchet', '名詞', 'B-product/weapon'),
 ('MS', '名詞', 'I-product/weapon'),
 ('フォント', '名詞', 'O'),
 ('で', '助詞', 'O'),
 ('は', '助詞', 'O'),
 ('、', '記号', 'O'),
 ('と', '助詞', 'O'),
 ('表示', '名詞', 'O'),
 ('さ', '動詞', 'O'),
 ('れ', '動詞', 'O'),
 ('"', '名詞', 'O'),
 ('et', '名詞', 'O'),
 ('"', '名詞', 'O'),
 ('の', '助詞', 'O'),
 ('合', '名詞', 'O'),
 ('字', '名詞', 'O'),
 ('で', '助動詞', 'O'),
 ('ある', '助動詞', 'O'),
 ('こと', '名詞', 'O'),
 ('が', '助詞', 'O'),


In [35]:
def word2features(doc, i):
    word = doc[i][0]
    postag = doc[i][1]

    # Common features for all words
    features = [
        'bias',
        'word.lower=' + word.lower(),
        'word[-3:]=' + word[-3:],
        'word[-2:]=' + word[-2:],
        'word.isupper=%s' % word.isupper(),
        'word.istitle=%s' % word.istitle(),
        'word.isdigit=%s' % word.isdigit(),
        'postag=' + postag
    ]

    # Features for words that are not
    # at the beginning of a document
    if i > 0:
        word1 = doc[i-1][0]
        postag1 = doc[i-1][1]
        features.extend([
            '-1:word.lower=' + word1.lower(),
            '-1:word.istitle=%s' % word1.istitle(),
            '-1:word.isupper=%s' % word1.isupper(),
            '-1:word.isdigit=%s' % word1.isdigit(),
            '-1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'beginning of a document'
        features.append('BOS')

    # Features for words that are not
    # at the end of a document
    if i < len(doc)-1:
        word1 = doc[i+1][0]
        postag1 = doc[i+1][1]
        features.extend([
            '+1:word.lower=' + word1.lower(),
            '+1:word.istitle=%s' % word1.istitle(),
            '+1:word.isupper=%s' % word1.isupper(),
            '+1:word.isdigit=%s' % word1.isdigit(),
            '+1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'end of a document'
        features.append('EOS')

    return features

In [36]:
from sklearn.model_selection import train_test_split

# A function for extracting features in documents
def extract_features(doc):
    return [word2features(doc, i) for i in range(len(doc))]

# A function fo generating the list of labels for each document
def get_labels(doc):
    return [label for (token, postag, label) in doc]

X = [extract_features(doc) for doc in data]
y = [get_labels(doc) for doc in data]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [37]:
import pycrfsuite
trainer = pycrfsuite.Trainer(verbose=True)

# Submit training data to the trainer
for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)

# Set the parameters of the model
trainer.set_params({
    # coefficient for L1 penalty
    'c1': 0.1,

    # coefficient for L2 penalty
    'c2': 0.01,  

    # maximum number of iterations
    'max_iterations': 200,

    # whether to include transitions that
    # are possible, but not observed
    'feature.possible_transitions': True
})

# Provide a file name as a parameter to the train function, such that
# the model will be saved to the file when training is finished
trainer.train('crf.model')

Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0...

.1....2..

..3...

.4....

5....6.

...

***** Iteration #190 *****
Loss: 40270.996716
Feature norm: 557.737941
Error norm: 1174.757836
Active features: 121670
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 66.882

***** Iteration #191 *****
Loss: 40267.090580
Feature norm: 557.696107
Error norm: 744.527540
Active features: 121589
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 64.864



***** Iteration #192 *****
Loss: 40265.055674
Feature norm: 557.918780
Error norm: 1161.531909
Active features: 121569
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 64.832

***** Iteration #193 *****
Loss: 40261.215526
Feature norm: 557.880683
Error norm: 726.395578
Active features: 121469
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 64.691



***** Iteration #194 *****
Loss: 40259.572596
Feature norm: 558.100369
Error norm: 1210.283378
Active features: 121451
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 64.756

***** Iteration #195 *****
Loss: 40255.693393
Feature norm: 558.066390
Error norm: 700.428679
Active features: 121362
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 64.896

***** Iteration #196 *****
Loss: 40253.965972
Feature norm: 558.279325
Error norm: 1156.868899
Active features: 121397
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 66.614



***** Iteration #197 *****
Loss: 40250.354119
Feature norm: 558.250999
Error norm: 635.150206
Active features: 121380
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 66.798

***** Iteration #198 *****
Loss: 40248.634020
Feature norm: 558.448835
Error norm: 1138.731480
Active features: 121392
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 66.352

***** Iteration #199 *****
Loss: 40245.225468
Feature norm: 558.420683
Error norm: 654.932772
Active features: 121390
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 64.875

***** Iteration #200 *****
Loss: 40244.075596
Feature norm: 558.618250
Error norm: 1242.795022
Active features: 121397
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 65.151



L-BFGS terminated with the maximum number of iterations
Total seconds required for training: 14360.192

Storing the model
Number of active features: 121397 (300304)
Number of active attributes: 53181 (138631)
Number of active labels: 274 (274)
Writing labels
Writing attributes
Writing feature references for transitions
Writing feature references for attributes
Seconds required: 0.332



In [38]:
tagger = pycrfsuite.Tagger()
tagger.open('crf.model')
y_pred = [tagger.tag(xseq) for xseq in X_test]

# Let's take a look at a random sample in the testing set
i = 12
for x, y in zip(y_pred[i], [x[1].split("=")[1] for x in X_test[i]]):
    print('{}\t{}'.format(y, x))

小池田	O
マヤ	O
（	O
こ	O
いけ	O
だ	O
ま	O
や	O
、	O
1969	B-timex/date
年	I-timex/date
5	B-timex/date
月	I-timex/date
4	I-timex/date
日	I-timex/date
-	O
）	O
は	O
、	O
日本	B-location/gpe/country
の	O
漫画	B-product/art/book
家	I-product/art/book
。	O
山口	B-location/gpe/province
県	I-location/gpe/province
光	B-location/gpe/city
市	I-location/gpe/city
虹	B-location/gpe/city
ヶ	I-location/gpe/city
丘	I-location/gpe/city
生まれ	O
、	O
大阪	B-location/gpe/province
府	I-location/gpe/province
出身	O
。	O
京都市立芸術大学	O
版画	O
科	O
卒業	O
。	O
女性	O
。	O


In [39]:
def get_entities(seq):
    """Gets entities from sequence.
    Args:
        seq (list): sequence of labels.
    Returns:
        list: list of (chunk_type, chunk_start, chunk_end).
    Example:
        >>> seq = ['B-PER', 'I-PER', 'O', 'B-LOC']
        >>> print(get_entities(seq))
        [('PER', 0, 2), ('LOC', 3, 4)]
    """
    i = 0
    chunks = []
    seq = seq + ['O']  # add sentinel
    types = [tag.split('-')[-1] for tag in seq]
    while i < len(seq):
        if seq[i].startswith('B'):
            for j in range(i+1, len(seq)):
                if seq[j].startswith('I') and types[j] == types[i]:
                    continue
                break
            chunks.append((types[i], i, j))
            i = j
        else:
            i += 1
    return chunks


def f1_score(y_true, y_pred, sequence_lengths):
    """Evaluates f1 score.
    Args:
        y_true (list): true labels.
        y_pred (list): predicted labels.
        sequence_lengths (list): sequence lengths.
    Returns:
        float: f1 score.
    Example:
        >>> y_true = []
        >>> y_pred = []
        >>> sequence_lengths = []
        >>> print(f1_score(y_true, y_pred, sequence_lengths))
        0.8
    """
    correct_preds, total_correct, total_preds = 0., 0., 0.
    for lab, lab_pred, length in zip(y_true, y_pred, sequence_lengths):
        lab = lab[:length]
        lab_pred = lab_pred[:length]

        lab_chunks = set(get_entities(lab))
        lab_pred_chunks = set(get_entities(lab_pred))

        correct_preds += len(lab_chunks & lab_pred_chunks)
        total_preds += len(lab_pred_chunks)
        total_correct += len(lab_chunks)

    p = correct_preds / total_preds if correct_preds > 0 else 0
    r = correct_preds / total_correct if correct_preds > 0 else 0
    f1 = 2 * p * r / (p + r) if correct_preds > 0 else 0
    return f1

lengths = [len(y) for y in y_test]
f1_score(y_test, y_pred, lengths)

0.6474437410225035

In [40]:
len(tokenized_docs)

1078768